In [2]:
import pandas as pd
import os

In [7]:
dir_list = os.listdir('../data/total_data/')
dir_list

['동별_데이터_handling.xlsx',
 '부산지하철 역사별 행정동.csv',
 '역별 데이터_handling.xlsx',
 '지하철 역사 내 상가 임대 정보.csv',
 '출퇴근시간 유동인구_2021_월_handling.xlsx',
 '출퇴근시간 유동인구_2021_일_handling.xlsx',
 '행정동별_업장의_수_handling.csv']

In [ ]:
def readfile(file_name):
    f_type = file_name.split('.')[-1]
    if f_type == 'xlsx':
        return pd.read_excel(f'../data/total_data/{file_name}',index_col=0)
    elif f_type == 'csv':
        return pd.read_csv(f'../data/total_data/{file_name}',encoding='cp949',index_col=0)
    else:
        return None

readfile(dir_list[0])

In [12]:
dong_data = readfile(dir_list[0])
dong_store = readfile(dir_list[-1])

In [13]:
dong_data.head()

,구군별,동별,소계 레벨,1인가구수 레벨
0,중구,중앙동,1,3
1,중구,동광동,0,1
2,중구,대청동,2,3
3,중구,보수동,4,7
4,중구,부평동,1,2


In [14]:
dong_store.head()

,동 이름,업장 수 레벨
0,남천2동,7
1,당리동,9
2,광복동,10
3,전포1동,10
4,중앙동,10


In [16]:
dong_store.rename(columns={'동 이름':'동별'},inplace=True)
dong_store.head()

,동별,업장 수 레벨
0,남천2동,7
1,당리동,9
2,광복동,10
3,전포1동,10
4,중앙동,10


In [171]:
step_one = dong_data.join(dong_store.set_index('동별'),on='동별')
step_one.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 205 entries, 0 to 204
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   구군별       205 non-null    object 
 1   동별        205 non-null    object 
 2   소계 레벨     205 non-null    int64  
 3   1인가구수 레벨  205 non-null    int64  
 4   업장 수 레벨   204 non-null    float64
dtypes: float64(1), int64(2), object(2)
memory usage: 9.6+ KB


In [21]:
station_store = readfile(dir_list[3])
station_store.head()

,상가번호,역사명,상가명,업종,면적(㎡),연간계약금액(부가세포함),계약시작일,계약종료일,비고
0,9501,다대포해수욕장역,CU,브랜드 편의점,52.54,NaN,20170420.0,20220419.0,휴점
1,9502,다대포해수욕장역,상가2호,상가,15.00,15840000.0,20191203.0,20241202.0,NaN
2,9601,다대포항역,CU,브랜드 편의점,75.60,NaN,20170420.0,20220419.0,휴점
3,9701,낫개역,CU,브랜드 편의점,24.33,NaN,20170420.0,20220419.0,NaN
4,9801,신장림역,CU,브랜드 편의점,77.57,NaN,20170420.0,20220419.0,NaN


In [52]:
station_store_num = pd.DataFrame(columns=['역사명','상가 갯수','사용중인 상가','비어있는 상가'])

In [37]:
station_store.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 375 entries, 0 to 374
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   상가번호           375 non-null    object 
 1   역사명            375 non-null    object 
 2   상가명            375 non-null    object 
 3   업종             375 non-null    object 
 4   면적(㎡)          375 non-null    float64
 5   연간계약금액(부가세포함)  375 non-null    float64
 6   계약시작일          375 non-null    float64
 7   계약종료일          375 non-null    float64
 8   비고             375 non-null    object 
dtypes: float64(4), object(5)
memory usage: 37.4+ KB


##### 지하상가 전체 갯수와 비어있는 상가, 사용중인 상가 갯수를 나타내기 위한 데이터 프레임

In [24]:
# nan값을 0으로 대체하여 공실 여부 판단
station_store.fillna(0,inplace=True)

In [66]:
store_info = {}

for idx in range(len(station_store)):
    item = station_store.iloc[idx]
    if list(item)[1] in store_info.keys():
        store_info[list(item)[1]]['상가 갯수'] += 1
        if list(item)[6] != 0.0:
            store_info[list(item)[1]]['사용중인 상가'] += 1
        else:
            store_info[list(item)[1]]['비어있는 상가'] += 1
    else:
        store_info[list(item)[1]] = {'상가 갯수':1,'사용중인 상가':0,'비어있는 상가':0}
        if list(item)[6] != 0.0:
            store_info[list(item)[1]]['사용중인 상가'] += 1
        else:
            store_info[list(item)[1]]['비어있는 상가'] += 1

In [67]:
len(store_info.keys())

94

In [86]:
i = 0
for item in store_info.keys():
    station_store_num.loc[i] = [item[:-1]]+list(store_info[item].values())
    i+=1

In [88]:
station_store_num.head()

,역사명,상가 갯수,사용중인 상가,비어있는 상가
0,다대포해수욕장,2,2,0
1,다대포항,1,1,0
2,낫개,1,1,0
3,신장림,1,1,0
4,장림,1,1,0


In [89]:
station_store_num.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 94 entries, 0 to 93
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   역사명      94 non-null     object
 1   상가 갯수    94 non-null     int64 
 2   사용중인 상가  94 non-null     int64 
 3   비어있는 상가  94 non-null     int64 
dtypes: int64(3), object(1)
memory usage: 5.7+ KB


In [90]:
station_store_num[station_store_num['상가 갯수']>1]


,역사명,상가 갯수,사용중인 상가,비어있는 상가
0,다대포해수욕장,2,2,0
6,신평,9,6,3
7,하단,10,10,0
8,당리,4,2,2
9,토성,3,3,0
10,자갈치,16,15,1
11,중앙,25,23,2
12,부산,5,4,1
13,초량,10,7,3
14,부산진,3,3,0


In [138]:
station_dong = readfile(dir_list[1])
station_dong.head()

,역명,행정동
선명,,
1호선,다대포해수욕장,다대동
1호선,다대포항,다대동
1호선,낫개,다대동
1호선,신장림,장림동
1호선,장림,장림동


In [139]:
station_dong.reset_index(drop=True,inplace=True)
station_dong.rename(columns={'역명':'역사명'},inplace=True)
station_dong.head()

,역사명,행정동
0,다대포해수욕장,다대동
1,다대포항,다대동
2,낫개,다대동
3,신장림,장림동
4,장림,장림동


In [140]:
station_dong.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114 entries, 0 to 113
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   역사명     114 non-null    object
 1   행정동     114 non-null    object
dtypes: object(2)
memory usage: 1.9+ KB


In [153]:
station_data = readfile(dir_list[2])
station_data.head()

,호선,역사명,환승,역위도,역경도,하행 대기시간,상행 대기시간,일평균승차 레벨,일평균하차 레벨,일평균승하차 레벨,순위 레벨
0,1,다대포해수욕장역,NaN,35.048670,128.964100,20,0,4,4,4,6
1,1,다대포항역,NaN,35.057820,128.971300,20,20,2,3,3,7
2,1,낫개역,NaN,35.065265,128.979873,20,20,4,4,4,6
3,1,신장림역,NaN,35.074433,128.977041,20,20,2,2,2,8
4,1,장림역,NaN,35.081090,128.977500,20,20,2,2,2,8


In [113]:
import re

In [141]:
for idx in range(len(station_dong.역사명)):
    if '(' in station_dong.역사명.iloc[idx]:
        station_dong.역사명.iloc[idx] = re.split('\(',station_dong.역사명.iloc[idx])[0]

In [154]:
for idx in range(len(station_data.역사명)):
    station_data.역사명.iloc[idx] = station_data.역사명.iloc[idx][:-1]
station_data.head()

C:\Users\0\AppData\Local\Temp\ipykernel_19532\4229311338.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  station_data.역사명.iloc[idx] = station_data.역사명.iloc[idx][:-1]


,호선,역사명,환승,역위도,역경도,하행 대기시간,상행 대기시간,일평균승차 레벨,일평균하차 레벨,일평균승하차 레벨,순위 레벨
0,1,다대포해수욕장,NaN,35.048670,128.964100,20,0,4,4,4,6
1,1,다대포항,NaN,35.057820,128.971300,20,20,2,3,3,7
2,1,낫개,NaN,35.065265,128.979873,20,20,4,4,4,6
3,1,신장림,NaN,35.074433,128.977041,20,20,2,2,2,8
4,1,장림,NaN,35.081090,128.977500,20,20,2,2,2,8


In [155]:
print(list(station_dong.역사명))
print(list(station_data.역사명))

['다대포해수욕장', '다대포항', '낫개', '신장림', '장림', '동매', '신평', '하단', '당리', '사하', '괴정', '대티', '서대신', '동대신', '토성', '자갈치', '남포', '중앙', '부산', '초량', '부산진', '좌천', '범일', '범내골', '서면', '부전', '양정', '시청', '연산', '교대', '동래', '명륜', '온천장', '부산대', '장전', '구서', '두실', '남산', '범어사', '노포', '장산', '중동', '해운대', '동백', '벡스코', '센텀시티', '민락', '수영', '광안', '금련산', '남천', '경성대·부경대', '대연', '못골', '지게골', '문현', '국제금융센터·부산은행', '전포', '서면', '부암', '가야', '동의대', '개금', '냉정', '주례', '감전', '사상', '덕포', '모덕', '모라', '구남', '구명', '덕천', '수정', '화명', '율리', '동원', '금곡', '호포', '증산', '부산대양산캠퍼스', '남양산', '양산', '수영', '망미', '배산', '물만골', '연산', '거제', '종합운동장', '사직', '미남', '만덕', '남산정', '숙등', '덕천', '구포', '강서구청', '체육공원', '대저', '미남', '동래', '수안', '낙민', '충렬사', '명장', '서동', '금사', '반여농산물시장', '석대', '영산대', '동부산대학', '고촌', '안평']
['다대포해수욕장', '다대포항', '낫개', '신장림', '장림', '동매', '신평', '하단', '당리', '사하', '괴정', '대티', '서대신', '동대신', '토성', '자갈치', '남포', '중앙', '부산', '초량', '부산진', '좌천', '범일', '범내골', '서면', '부전', '양정', '시청', '연산', '교대', '동래', '명륜', '온천장', '부산대', '장전', '구서', '두실', '남산', '범어사', '

In [156]:
print(len(list(station_dong.역사명)))
print(len(list(station_data.역사명)))

114
112


In [164]:
step_two = station_dong.join(station_data.set_index('역사명'),on='역사명')
step_two

,역사명,행정동,호선,환승,역위도,역경도,하행 대기시간,상행 대기시간,일평균승차 레벨,일평균하차 레벨,일평균승하차 레벨,순위 레벨
0,다대포해수욕장,다대동,1,NaN,35.048670,128.964100,20,0,4,4,4,6
1,다대포항,다대동,1,NaN,35.057820,128.971300,20,20,2,3,3,7
2,낫개,다대동,1,NaN,35.065265,128.979873,20,20,4,4,4,6
3,신장림,장림동,1,NaN,35.074433,128.977041,20,20,2,2,2,8
4,장림,장림동,1,NaN,35.081090,128.977500,20,20,2,2,2,8
...,...,...,...,...,...,...,...,...,...,...,...,...
109,석대,석대동,4,NaN,35.218112,129.137179,20,20,0,0,0,10
110,영산대,반송동,4,NaN,35.225777,129.146149,20,20,2,2,2,8
111,동부산대학,반송동,4,NaN,35.232506,129.154024,20,20,1,1,1,9
112,고촌,철마면 고촌리,4,NaN,35.236031,129.160444,20,20,0,0,0,10


In [167]:
step_two = step_two.join(station_store_num.set_index('역사명'),on='역사명')
step_two.head()

,역사명,행정동,호선,환승,역위도,역경도,하행 대기시간,상행 대기시간,일평균승차 레벨,일평균하차 레벨,일평균승하차 레벨,순위 레벨,상가 갯수,사용중인 상가,비어있는 상가
0,다대포해수욕장,다대동,1,NaN,35.048670,128.964100,20,0,4,4,4,6,2.0,2.0,0.0
1,다대포항,다대동,1,NaN,35.057820,128.971300,20,20,2,3,3,7,1.0,1.0,0.0
2,낫개,다대동,1,NaN,35.065265,128.979873,20,20,4,4,4,6,1.0,1.0,0.0
3,신장림,장림동,1,NaN,35.074433,128.977041,20,20,2,2,2,8,1.0,1.0,0.0
4,장림,장림동,1,NaN,35.081090,128.977500,20,20,2,2,2,8,1.0,1.0,0.0


In [172]:
step_one.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 205 entries, 0 to 204
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   구군별       205 non-null    object 
 1   동별        205 non-null    object 
 2   소계 레벨     205 non-null    int64  
 3   1인가구수 레벨  205 non-null    int64  
 4   업장 수 레벨   204 non-null    float64
dtypes: float64(1), int64(2), object(2)
memory usage: 9.6+ KB


In [173]:
step_two.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 122 entries, 0 to 113
Data columns (total 15 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   역사명        122 non-null    object 
 1   행정동        122 non-null    object 
 2   호선         122 non-null    int64  
 3   환승         20 non-null     float64
 4   역위도        122 non-null    float64
 5   역경도        122 non-null    float64
 6   하행 대기시간    122 non-null    int64  
 7   상행 대기시간    122 non-null    int64  
 8   일평균승차 레벨   122 non-null    int64  
 9   일평균하차 레벨   122 non-null    int64  
 10  일평균승하차 레벨  122 non-null    int64  
 11  순위 레벨      122 non-null    int64  
 12  상가 갯수      81 non-null     float64
 13  사용중인 상가    81 non-null     float64
 14  비어있는 상가    81 non-null     float64
dtypes: float64(6), int64(7), object(2)
memory usage: 15.2+ KB


In [174]:
step_two.rename(columns={'행정동':'동별'},inplace=True)

In [177]:
final_dataframe = step_one.join(step_two.set_index('동별'),on='동별')

In [178]:
final_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 210 entries, 0 to 204
Data columns (total 19 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   구군별        210 non-null    object 
 1   동별         210 non-null    object 
 2   소계 레벨      210 non-null    int64  
 3   1인가구수 레벨   210 non-null    int64  
 4   업장 수 레벨    209 non-null    float64
 5   역사명        15 non-null     object 
 6   호선         15 non-null     float64
 7   환승         0 non-null      float64
 8   역위도        15 non-null     float64
 9   역경도        15 non-null     float64
 10  하행 대기시간    15 non-null     float64
 11  상행 대기시간    15 non-null     float64
 12  일평균승차 레벨   15 non-null     float64
 13  일평균하차 레벨   15 non-null     float64
 14  일평균승하차 레벨  15 non-null     float64
 15  순위 레벨      15 non-null     float64
 16  상가 갯수      10 non-null     float64
 17  사용중인 상가    10 non-null     float64
 18  비어있는 상가    10 non-null     float64
dtypes: float64(14), int64(2), object(3)
memory usage: 

In [180]:
print(list(step_one.동별))
print(list(step_two.동별))

['중앙동', '동광동', '대청동', '보수동', '부평동', '광복동', '남포동', '영주1동', '영주2동', '동대신1동', '동대신2동', '동대신3동', '서대신1동', '서대신3동', '서대신4동', '부민동', '아미동', '초장동', '충무동', '남부민1동', '남부민2동', '암남동', '초량1동', '초량2동', '초량3동', '초량6동', '수정1동', '수정2동', '수정4동', '수정5동', '좌천동', '범일1동', '범일2동', '범일5동', '남항동', '영선1동', '영선2동', '신선동', '봉래1동', '봉래2동', '청학1동', '청학2동', '동삼1동', '동삼2동', '동삼3동', '부전1동', '부전2동', '연지동', '초읍동', '양정1동', '양정2동', '전포1동', '전포2동', '부암1동', '부암3동', '당감1동', '당감2동', '당감4동', '가야1동', '가야2동', '개금1동', '개금2동', '개금3동', '범천1동', '범천2동', '수민동', '복산동', '명륜동', '온천1동', '온천2동', '온천3동', '사직1동', '사직2동', '사직3동', '안락1동', '안락2동', '명장1동', '명장2동', '대연1동', '대연3동', '대연4동', '대연5동', '대연6동', '용호1동', '용호2동', '용호3동', '용호4동', '용당동', '감만1동', '감만2동', '우암동', '문현1동', '문현2동', '문현3동', '문현4동', '구포1동', '구포2동', '구포3동', '금곡동', '화명1동', '화명2동', '화명3동', '덕천1동', '덕천2동', '덕천3동', '만덕1동', '만덕2동', '만덕3동', '우1동', '우2동', '우3동', '중1동', '중2동', '좌1동', '좌2동', '좌3동', '좌4동', '송정동', '반여1동', '반여2동', '반여3동', '반여4동', '반송1동', '반송2동', '재송1동', '재송2동', '괴정1동', '괴정2동', 

In [179]:
final_dataframe.to_csv('../data/total_data/데이터_합본.csv',encoding='cp949')

In [181]:
step_one.to_csv('../data/total_data/동_데이터_합본.csv',encoding='cp949')
step_two.to_csv('../data/total_data/역_데이터_합본.csv',encoding='cp949')

In [208]:
new_step_two = pd.read_csv('../data/total_data/역_데이터_합본.csv',encoding='cp949',index_col=0)
new_step_two.head()

,역사명,동별,호선,환승,역위도,역경도,하행 대기시간,상행 대기시간,일평균승차 레벨,일평균하차 레벨,일평균승하차 레벨,순위 레벨,상가 갯수,사용중인 상가,비어있는 상가
0,다대포해수욕장,다대1동,1,NaN,35.048670,128.964100,20,0,4,4,4,6,2.0,2.0,0.0
1,다대포항,다대1동,1,NaN,35.057820,128.971300,20,20,2,3,3,7,1.0,1.0,0.0
2,낫개,다대2동,1,NaN,35.065265,128.979873,20,20,4,4,4,6,1.0,1.0,0.0
3,신장림,장림2동,1,NaN,35.074433,128.977041,20,20,2,2,2,8,1.0,1.0,0.0
4,장림,장림2동,1,NaN,35.081090,128.977500,20,20,2,2,2,8,1.0,1.0,0.0


In [209]:
total_df = new_step_two.join(step_one.set_index('동별'),on='동별')
total_df.head()

,역사명,동별,호선,환승,역위도,역경도,하행 대기시간,상행 대기시간,일평균승차 레벨,일평균하차 레벨,일평균승하차 레벨,순위 레벨,상가 갯수,사용중인 상가,비어있는 상가,구군별,소계 레벨,1인가구수 레벨,업장 수 레벨
0,다대포해수욕장,다대1동,1,NaN,35.048670,128.964100,20,0,4,4,4,6,2.0,2.0,0.0,사하구,10.0,9.0,8.0
1,다대포항,다대1동,1,NaN,35.057820,128.971300,20,20,2,3,3,7,1.0,1.0,0.0,사하구,10.0,9.0,8.0
2,낫개,다대2동,1,NaN,35.065265,128.979873,20,20,4,4,4,6,1.0,1.0,0.0,사하구,9.0,8.0,7.0
3,신장림,장림2동,1,NaN,35.074433,128.977041,20,20,2,2,2,8,1.0,1.0,0.0,사하구,8.0,8.0,8.0
4,장림,장림2동,1,NaN,35.081090,128.977500,20,20,2,2,2,8,1.0,1.0,0.0,사하구,8.0,8.0,8.0


In [210]:
total_df.to_csv('../data/total_data/데이터_합본.csv',encoding='cp949')